In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from pymongo import MongoClient
from deep_translator import GoogleTranslator

# Definicion de Funciones

In [3]:
def query_mongodb(collection: str = '', query_json: dict = {}, query_project: dict = {}, limit: int = 0):
    # Define MongoDB parameters
    mongo_srv_uri = 'mongodb://localhost:27017/'
    mongo_user = ''
    mongo_password = ''
    mongo_db_name = 'facsat2'
    mongo_collection_name = collection

    try:
        # Connect to MongoDB
        client = MongoClient("mongodb://localhost:27017/", connectTimeoutMS=30000)
    
        # Access the MongoDB database
        db = client[mongo_db_name]
        collection = db[mongo_collection_name]

        # Perform MongoDB operations
        result = collection.find(query_json, query_project).limit(limit) 
    except Exception as e:
        print("No fue posible realizar la consulta\nError:\n")
        print(e)
    
    data = list(result) # pd.DataFrame(list(result))
    return data

In [4]:
def crear_dataframe(data):
    df = pd.DataFrame(data)  # Crear un DataFrame inicial

    # Función para crear nuevas columnas basadas en 'Param.Name'
    def crear_columna(row):
        new_row = row.copy()
        new_row[row['Param']['Name']] = row['Val']
        del new_row['Val']
        # Agregar las columnas 'Node' y 'Table'
        new_row['Node'] = row['Param']['Node']
        new_row['Table'] = row['Param']['Table']
        new_row['Index'] = row['Param']['Index'] if 'Index' in row['Param'] else np.nan
        
        return new_row

    # Aplicar la función a cada fila y crear un nuevo DataFrame
    df = df.apply(crear_columna, axis=1)

    # Eliminar la columna 'Param' ya que sus valores han sido distribuidos
    df = df.drop('Param', axis=1)

    return df

In [33]:
def traducir_texto(cadena):
    traducido = GoogleTranslator(source='en', target='es').translate(cadena)
    return traducido

# Función para verificar si un número es entero
def es_entero(num):
    if np.all(num == num.astype(int)):
        tipo = 'int'
    elif np.all(num == num.astype(float)):
        tipo = 'float'
    else:
        'No Identificado'
    return tipo

def crear_diccionario_de_datos(data: dict = {}, info: dict = {}):

    data_df = pd.json_normalize(data)
    info_df = pd.json_normalize(info)
    
    try:
        tipo_dato = data_df.copy()
        tipo_dato['Tipo de Dato'] = tipo_dato.groupby(by=['Param.Name', 'Param.Node', 'Param.Table'], as_index=False)['Val'].transform(es_entero)
        tipo_dato = tipo_dato.groupby(by=['Param.Name', 'Param.Node', 'Param.Table'])['Tipo de Dato'].first()
        # Eliminar columnas que no son necesarias
        data_df = data_df.drop(['_id', 'Ts', 'Val', 'Param.Satellite'], axis=1)
        data_df.drop_duplicates(inplace=True, ignore_index=True)
        info_df = info_df.drop(['_id', 'Param.Satellite'], axis=1)
        info_df['HelpText'] = info_df['HelpText'].apply(traducir_texto)
        
        # Hacer merge de dataframe info_df con data_df
        data_df = data_df.merge(info_df, on=['Param.Node', 'Param.Table', 'Param.Name'], how='left')
        data_df = data_df.merge(tipo_dato, on=['Param.Node', 'Param.Table', 'Param.Name'], how='left')
        
    except Exception as e:
        print(e)

    return data_df

# Consulta a la base de datos MongoDB

In [6]:
ultima_fecha = datetime(2024, 8, 25, 0, 0)
un_mes_atras = ultima_fecha - timedelta(days=1)
timestamp_mes_atras = int(un_mes_atras.timestamp())

In [7]:
acronimos = {
  "ADCS": {
    "significado": "Attitude Determination and Control System",
    "descripcion": "Sistema responsable de la determinación y control de la actitud del satélite, utilizando sensores y actuadores para orientar el satélite en el espacio."
  },
  "OBC": {
    "significado": "On-Board Computer",
    "descripcion": "Computador de a bordo que gestiona las operaciones del satélite y el procesamiento de datos."
  },
  "EPS": {
    "significado": "Electrical Power System",
    "descripcion": "Sistema encargado de la gestión de la energía eléctrica del satélite, incluyendo la distribución y el almacenamiento en baterías."
  },
  "PDU": {
    "significado": "Power Distribution Unit",
    "descripcion": "Unidad responsable de distribuir energía a los diferentes subsistemas del satélite."
  },
  "BPX": {
    "significado": "Battery Pack",
    "descripcion": "Paquete de baterías que almacena la energía para el funcionamiento del satélite."
  },
  "FSS": {
    "significado": "Fine Sun Sensor",
    "descripcion": "Sensor solar fino que detecta la posición del sol para ajustar la orientación del satélite."
  },
  "CSS": {
    "significado": "Coarse Sun Sensor",
    "descripcion": "Sensor solar grueso utilizado para determinar la posición general del sol."
  },
  "MS100": {
    "significado": "MultiScape 100",
    "descripcion": "Sensor de la carga útil del satélite que captura imágenes o datos relacionados con la misión."
  },
  "A2K": {
    "significado": "Argus 2000",
    "descripcion": "Sensor avanzado utilizado en la carga útil del satélite para la adquisición de datos científicos."
  },
  "CSP": {
    "significado": "CubeSat Space Protocol",
    "descripcion": "Protocolo de comunicaciones utilizado para interconectar los sistemas a bordo de CubeSats."
  },
  "RW": {
    "significado": "Reaction Wheel",
    "descripcion": "Rueda de reacción utilizada para controlar la orientación del satélite sin necesidad de utilizar propulsores."
  },
  "AFE": {
    "significado": "Analog Front-End",
    "descripcion": "Interfaz analógica utilizada para la conversión y procesamiento de señales analógicas en el satélite."
  },
  "SDR": {
    "significado": "Software Defined Radio",
    "descripcion": "Radio definida por software que permite la reconfiguración y modulación de señales de comunicación."
  },
  "TLE": {
    "significado": "Two-Line Element Set",
    "descripcion": "Conjunto de dos líneas que contienen los parámetros orbitales necesarios para rastrear el satélite."
  },
  "GPS": {
    "significado": "Global Positioning System",
    "descripcion": "Sistema de posicionamiento global utilizado para determinar la posición y velocidad del satélite."
  },
  "I2C": {
    "significado": "Inter-Integrated Circuit",
    "descripcion": "Protocolo de comunicación entre circuitos integrados que permite la transmisión de datos entre dispositivos del satélite."
  },
  "UART": {
    "significado": "Universal Asynchronous Receiver-Transmitter",
    "descripcion": "Receptor-transmisor asincrónico universal utilizado para la transmisión de datos serie en el satélite."
  },
  "HK": {
    "significado": "Housekeeping",
    "descripcion": "Telemetría básica que monitorea las condiciones esenciales del satélite, como temperatura y energía."
  },
  "XT8250": {
    "significado": "XT8250",
    "descripcion": "Modelo de procesador o sistema utilizado en el satélite para el control de la carga útil."
  },
  "DVBS2": {
    "significado": "Digital Video Broadcasting - Satellite Second Generation",
    "descripcion": "Sistema de modulación digital utilizado para la transmisión de datos a través de satélites de segunda generación."
  }
}


In [8]:
nodos = {
    "OBC": 1,
    "ADCS": 4,
    "AX2150": 5,
    "P60 DOCK": 6,
    "P60 PDU 1": 7,
    "P60 PDU 2": 8,
    "P60 ACU": 10,
    "XT8250 MONITOR": 12,
    "mddvbs2-control-app": 14,
    "BPX 1": 16,
    "BPX 2": 17,
    "Payload App": 20,
    "Payload OBC MONITOR": 22,
    "AFE8250": 23
  }

In [11]:
node_6 = query_mongodb('ParamData', {"Param.Node": 4, "Ts": { "$gte": timestamp_mes_atras}}, limit=0)
node_6_info = query_mongodb('ParamInfo', {"Param.Node": 4}, {"HelpText": 1, "Param": 1 }, limit=0)

In [31]:
a = crear_diccionario_de_datos(node_6, node_6_info)

In [34]:
dataframe_final = pd.DataFrame()

for clave, valor in nodos.items():
    print(f"Nodo evaluado: {clave}: {valor}")
    node = query_mongodb('ParamData', {"Param.Node": valor, "Ts": { "$gte": timestamp_mes_atras}}, limit=0)
    node_info = query_mongodb('ParamInfo', {"Param.Node": valor}, {"HelpText": 1, "Param": 1 }, limit=0)
    diccionario = crear_diccionario_de_datos(node, node_info)
    diccionario.insert(0, 'Sistema', [clave]*len(diccionario))
    dataframe_final = pd.concat([dataframe_final, diccionario])

dataframe_final["Param.Node"] = dataframe_final["Param.Node"].astype(int)
dataframe_final.rename(columns={"Param.Node": "Nodo", "Param.Index": "Canal/Item", "Param.Name": "Variable", "Param.Table": "Tabla Beacon", "HelpText": "Descripcion"}, inplace=True)
dataframe_final.to_csv("/outputs/diccionario_de_datos.xlsx", index=False)

Nodo evaluado: OBC: 1
Nodo evaluado: ADCS: 4
Nodo evaluado: AX2150: 5
Nodo evaluado: P60 DOCK: 6
Nodo evaluado: P60 PDU 1: 7
Nodo evaluado: P60 PDU 2: 8
Nodo evaluado: P60 ACU: 10
Nodo evaluado: XT8250 MONITOR: 12
'Param.Name'
Nodo evaluado: mddvbs2-control-app: 14
Nodo evaluado: BPX 1: 16
'Param.Name'
Nodo evaluado: BPX 2: 17
'Param.Name'
Nodo evaluado: Payload App: 20
Nodo evaluado: Payload OBC MONITOR: 22
'Param.Name'
Nodo evaluado: AFE8250: 23


In [29]:
dataframe_final.describe()

,Param.Node,Param.Table,Param.Index
count,171900.000000,171900.000000,121041.000000
mean,6.675422,50.357627,3.692393
std,3.807787,67.273079,3.599921
min,1.000000,1.000000,0.000000
25%,4.000000,4.000000,1.000000
50%,6.000000,4.000000,3.000000
75%,7.000000,150.000000,6.000000
max,23.000000,156.000000,23.000000


In [40]:
acronimos = pd.DataFrame.from_dict(acronimos, orient='index')

In [48]:
acronimos.to_csv("/outputs/tabla_acronimos.xlsx", index_label="Acronimo")